## librerie

In [1]:
import pandas as pd
import numpy as np
import tensorflow
import copy
import re
import string
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.feature_extraction.text  import TfidfVectorizer
import string as s
import community
from community import community_louvain
import networkx as nx

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# dati

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
vax=pd.read_csv("/content/drive/MyDrive/SMA/vax2.csv", sep=",")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#verifico che non ci siano retweet
vax_noret=vax[vax['is_RT']==False]

In [5]:
#seleziono solo i tweet da UK
vax['location']=vax.location.str.lower()
vax_uk=vax[vax.location.str.contains('uk|england|united kingdom|london' , na=False)]
#vax_uk=vax[vax.location=="uk"]

In [6]:
#filtro righe che non hanno text e username vuoto
vax_uk=vax_uk[vax_uk['text'].notna()]

In [7]:
len(vax_uk)

26466

# preprocessing per analisi grafo

In [8]:
pp=vax_uk.text[:]

In [9]:
#tokenization
def tokenization(text):
    lst=text.split()
    return lst

In [10]:
pp=pp.apply(tokenization)

In [11]:
#conversion lowercase
def lowercasing(lst):
    new_lst=[]
    for  i in  lst:
        i=i.lower()
        new_lst.append(i) 
    return new_lst

In [12]:
puntegg=s.punctuation.replace(s.punctuation[2], '') #no #
puntegg=puntegg.replace(s.punctuation[21], '') #no @
puntegg

'!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~'

In [13]:
#punctuation (tranne cancelletto e chiocciola)
def remove_punctuations(lst):
    new_lst=[]
    for i in lst:
        for  j in  puntegg:
            i=i.replace(j,'')
        new_lst.append(i)
    return new_lst

In [14]:
pp=pp.apply(remove_punctuations)

In [15]:
pp=pp.apply(lambda x: ''.join(i+' ' for i in x))

In [16]:
vax_uk['text']=pp

# estrazione mentions e hashtag

In [17]:
#estraggo mention
def find_mentions(x):
    p = re.compile(r'@([^\s:]+)')
    return ','.join(p.findall(x))

vax_uk["@mentions"] = vax_uk["text"].apply(find_mentions)

In [18]:
#estraggo hashtag
def find_hashtags(x):
    p = re.compile(r'#([^\s:]+)')
    return ','.join(p.findall(x))

vax_uk["#hashtags"] = vax_uk["text"].apply(find_hashtags)

In [19]:
relations = vax_uk[["user_id", "username","@mentions","#hashtags"]]

MENZIONI

In [20]:
rel_mentions=relations[(relations["@mentions"]!="")]

In [21]:
rel_mentions["splitted_mentions"]=rel_mentions["@mentions"].apply(lambda x: x.split(','))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
def remove_comma(x):
    filtered = [i for i in x if i.strip()]
    return filtered

In [23]:
rel_mentions["splitted_mentions"]=rel_mentions["splitted_mentions"].apply(remove_comma)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
###### ANALISI DELLE MENZIONI ###########
menzioni = pd.DataFrame(rel_mentions["@mentions"].str.split(",",expand=True))
menzioni=menzioni.melt()
menzioni['value']=menzioni.value.str.strip()
menzioni = menzioni[pd.notnull(menzioni['value'])]
menzioni = pd.DataFrame(menzioni["value"].value_counts()).reset_index()
menzioni.rename(columns = {'index': "Persone citate", 
                           'value':"Numero di citazioni"}, inplace = True)
#mentions_count = pd.DataFrame(menzioni[menzioni["Numero di citazioni"] > 5]).reset_index()
mentions_count = pd.DataFrame(menzioni[menzioni["Numero di citazioni"] > 1]).reset_index()

In [25]:
mentions_count.head(10)

,index,Persone citate,Numero di citazioni
0,0,matthancock,679
1,1,borisjohnson,383
2,2,gmb,272
3,3,piersmorgan,268
4,4,nhsengland,249
5,5,skynews,232
6,6,realdonaldtrump,177
7,7,juliahb1,173
8,8,pfizer,156
9,9,nhsuhcw,149


In [26]:
################# ANALISI UTENTI CHE HANNO MENZIONATO ###############

users=rel_mentions.groupby(['username']).count().sort_values(['user_id'], ascending=False)

users.head(10)

,user_id,@mentions,#hashtags,splitted_mentions
username,,,,
FintechZoom,100,100,100,100
Ali #KBF,39,39,39,39
David E Brady,34,34,34,34
Moira,30,30,30,30
noreen,29,29,29,29
Henners,29,29,29,29
Vincent Petrucci,28,28,28,28
Jasdip Sagu,26,26,26,26
Chris Moyler,25,25,25,25


HASHTAGS

In [27]:
rel_hashtag=relations[(relations["#hashtags"]!="")]

In [28]:
rel_hashtag["splitted_hashtags"] = rel_hashtag["#hashtags"].apply(lambda x: x.split(','))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
def from_list_to_nan(x):
    if x == ['']:
        x = None
        return x
    else:
        return x

In [30]:
rel_hashtag["splitted_hashtags"] =rel_hashtag["splitted_hashtags"].apply(from_list_to_nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
###### ANALISI DELLE MENZIONI ###########
hashtag= pd.DataFrame(rel_hashtag["#hashtags"].str.split(",",expand=True))
hashtag=hashtag.melt()
hashtag['value']=hashtag.value.str.strip()
hashtag = hashtag[pd.notnull(hashtag['value'])]
hashtag = pd.DataFrame(hashtag["value"].value_counts()).reset_index()
hashtag.rename(columns = {'index': "Hashtag", 
                           'value':"Numero di hashtag"}, inplace = True)
#hashtag_count = pd.DataFrame(hashtag[hashtag["Numero di hashtag"] > 5]).reset_index()
hashtag_count = pd.DataFrame(hashtag[hashtag["Numero di hashtag"] > 1]).reset_index()

In [32]:
hashtag_count.head(10)

,index,Hashtag,Numero di hashtag
0,0,covid19,1075
1,1,vaccine,1006
2,2,covidvaccine,598
3,3,covid19vaccine,260
4,4,coronavirus,204
5,5,covid,185
6,6,nhs,178
7,7,vaccines,143
8,8,vaccination,138
9,9,pfizer,128


# grafo menzioni


**GRAFO SULLA COMMUNITY GLOBALE UK**

In [33]:
G = nx.DiGraph()
for r in rel_mentions.iterrows():
    for user in r[1]['splitted_mentions']:
        G.add_edge(r[1]['username'], user)

In [ ]:
#nx.write_gexf(G, path="/content/drive/MyDrive/SMA/g_dir_uk.gexf")

In [ ]:
print(f"Ci sono {G.number_of_nodes()} nodi e {G.number_of_edges()} archi nel grafo")

Ci sono 18279 nodi e 20022 archi nel grafo


In [ ]:
in_degrees = [val for (node, val) in G.in_degree()]
print(f"in degree massimo del grafo è {np.max(in_degrees)}")   
print(f"in degree massimo del grafo è {np.min(in_degrees)}")
print(f"in degree medio del grafo è {np.mean(in_degrees)}")

in degree massimo del grafo è 523
in degree massimo del grafo è 0
in degree medio del grafo è 1.095355325783686


In [ ]:
out_degrees = [val for (node, val) in G.out_degree()]
print(f"out degree massimo del grafo è {np.max(out_degrees)}")   
print(f"out degree massimo del grafo è {np.min(out_degrees)}")
print(f"out degree medio del grafo è {np.mean(out_degrees)}")

out degree massimo del grafo è 82
out degree massimo del grafo è 0
out degree medio del grafo è 1.095355325783686


In [ ]:
hubs, score =nx.hits(G, max_iter=50,normalized=True)
  
print("Hub Scores: ", hubs) 
print("Authority Scores: ", score)

Hub Scores:  {'Dee Smith #SackVallance #SackWhitty #TrumpWon': 6.564516076646844e-110, 'xx': 0.0, 'Geordieonline': 8.86768781783036e-92, 'aetiology': 0.0, 'LoopyLou': 6.564516076646844e-110, 'chillyzag': 0.0, 'Yvonne 🐈🇬🇧': 1.8463391988014587e-06, 'eath1223': 0.0, 'Sam': 0.00099345355570495, 'leahbutlersmith': 0.0, 'simondolan': 0.0, 'Smoke': 1.8044404392489232e-98, 'daveyb0072': 0.0, 'FintechZoom': 6.564516076646844e-110, 'fintechzoom': 0.0, 'Tony': 0.0017210313542280408, 'nicolasturgeon': 0.0, 'ShellSears': 1.8044404392489232e-98, 'deanstrachan1': 0.0, 'sophiamullings': 0.0, 'David Brown': 0.00042524277430442596, 'todmartinn': 0.0, '@Emma Peel UK🍀🍾🕊🌍🦋✈': 1.8132027038567496e-05, 'dailymailuk': 0.0, 'Stephen Wright': 1.8044404392489232e-98, 'julianheap': 0.0, 'ellerbydave': 0.0, 'Stacey 🇬🇧#KBF 🇺🇸💪 Animal Lover': 0.0012798382831862183, 'matthancock': 0.0, 'pfizer': 0.0, 'biontechgroup': 0.0, 'UK Lockdown #HandsFaceSpace': 9.145881999327409e-06, 'ukcovid19stats': 0.0, 'Rob List': 2.388157

In [ ]:
print(f"il coefficente di assortativity {nx.degree_assortativity_coefficient(G)}")

il coefficente di assortativity -0.07590918563021193


In [ ]:
print(f"il coefficente di reciprocity è {nx.reciprocity(G)}")

il coefficente di reciprocity è 0.0


**boris jhonson e match hancock**

In [34]:
vax_uk2=vax_uk[vax_uk['text'].str.contains('matthancock|borisjohnson', case=False)]

In [35]:
#estraggo mention
def find_mentions(x):
    p = re.compile(r'@([^\s:]+)')
    return ','.join(p.findall(x))

vax_uk2["@mentions"] = vax_uk2["text"].apply(find_mentions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#estraggo hashtag
def find_hashtags(x):
    p = re.compile(r'#([^\s:]+)')
    return ', '.join(p.findall(x))

vax_uk2["#hashtags"] = vax_uk2["text"].apply(find_hashtags)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
len(vax_uk2)

1049

In [ ]:
relations = vax_uk2[["user_id", "username","@mentions","#hashtags"]]

In [ ]:
rel_mentions=relations[(relations["@mentions"]!="")]

In [ ]:
rel_mentions["splitted_mentions"]=rel_mentions["@mentions"].apply(lambda x: x.split(','))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def remove_comma(x):
    filtered = [i for i in x if i.strip()]
    return filtered

In [ ]:
rel_mentions["splitted_mentions"]=rel_mentions["splitted_mentions"].apply(remove_comma)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
G2 = nx.DiGraph()
for r in rel_mentions.iterrows():
    for user in r[1]['splitted_mentions']:
        G2.add_edge(r[1]['username'], user)

In [ ]:
#nx.write_gexf(G2, path="/content/drive/MyDrive/SMA/g_dir_jhonson.gexf")

In [ ]:
#in_degrees = [val for (node, val) in G2.in_degree()]

In [ ]:
in_degrees2=pd.DataFrame(G2.in_degree())

In [ ]:
plt.figure(figsize=(30, 30))
nx.draw_networkx(G2, cmap=plt.get_cmap('jet'),  
               node_size=(in_degrees2[1]+1)*10, 
               with_labels=False)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print(f"Ci sono {G2.number_of_nodes()} nodi e {G2.number_of_edges()} archi nel grafo")

Ci sono 1397 nodi e 2112 archi nel grafo


In [ ]:
in_deg = [val for (node, val) in G2.in_degree()]
print(f"in degree massimo del grafo è {np.max(in_deg)}")   
print(f"in degree massimo del grafo è {np.min(in_deg)}")

in degree massimo del grafo è 523
in degree massimo del grafo è 0


In [ ]:
out_degrees = [val for (node, val) in G2.out_degree()]
print(f"out degree massimo del grafo è {np.max(out_degrees)}")   
print(f"out degree massimo del grafo è {np.min(out_degrees)}")

out degree massimo del grafo è 26
out degree massimo del grafo è 0


In [ ]:
hubs, score =nx.hits(G2, max_iter=50,normalized=True)
  
print("Hub Scores: ", hubs) 
print("Authority Scores: ", score)

Hub Scores:  {'Stacey 🇬🇧#KBF 🇺🇸💪 Animal Lover': 0.0016695510723952448, 'matthancock': 0.0, 'pfizer': 0.0, 'biontechgroup': 0.0, 'REC': 0.0006170370178230848, 'leswee': 0.0, 'katewilton1': 0.0, 'borisjohnson': 0.0, 'Matt ash': 0.0016840203592912765, 'skillralph': 0.0, 'Malthi': 0.001776176505195649, 'mhragovuk': 0.0, 'Nicola Foale': 0.0020842580867603336, 'borisjohnsonmp': 0.0, 'realdonaldtrump': 0.0, '🎄Dan2020🎄': 0.001530667789421565, 'bronwen6720': 0.0, 'nadhimzahawi': 0.0, 'hugo kitchen': 0.0020724655702397095, 'honestyistheb11': 0.0, 'dazgraham22': 0.0, 'himupnorth': 0.0, 'caveman1975': 0.0, 'ITV plc': 0.000565443614857751, 'mr': 0.0, 'usa': 0.0, 'vp': 0.0, 'donaldtrump': 0.0, 'joebiden': 0.0, 'Iron Felix': 0.0005586169750082353, 'Peter': 0.0024371962163951097, 'rishisunak': 0.0, 'mpiainds': 0.0, 'daviddavismp': 0.0, 'cgreenuk': 0.0, 'desmondswayne': 0.0, '10downingstreet': 0.0, 'carriesymonds': 0.0, 'keirstarmer': 0.0, 'annasoubry': 0.0, 'edwardjdavey': 0.0, 'lukejohnsonrcp': 0.0, 

In [ ]:
print(f"il coefficente di assortativity {nx.degree_assortativity_coefficient(G2)}")

il coefficente di assortativity -0.3500517301253774


In [ ]:
print(f"il coefficente di reciprocity è {nx.reciprocity(G2)}")

il coefficente di reciprocity è 0.0


**cnn vs sky**

In [36]:
vax_uk33=vax_uk[vax_uk['text'].str.contains('cnn|sky', case=False)]

In [ ]:
len(vax_uk33)

421

In [37]:
vax_uk3=vax_uk[vax_uk['text'].str.contains('cnn|sky|braking|news', case=False)]

In [38]:
len(vax_uk3)

1862

In [39]:
#estraggo mention
def find_mentions(x):
    p = re.compile(r'@([^\s:]+)')
    return ','.join(p.findall(x))

vax_uk3["@mentions"] = vax_uk3["text"].apply(find_mentions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
#estraggo hashtag
def find_hashtags(x):
    p = re.compile(r'#([^\s:]+)')
    return ', '.join(p.findall(x))

vax_uk3["#hashtags"] = vax_uk3["text"].apply(find_hashtags)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
relations = vax_uk3[["user_id", "username","@mentions","#hashtags"]]

In [42]:
rel_mentions=relations[(relations["@mentions"]!="")]

In [43]:
rel_mentions["splitted_mentions"]=rel_mentions["@mentions"].apply(lambda x: x.split(','))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
def remove_comma(x):
    filtered = [i for i in x if i.strip()]
    return filtered

In [45]:
rel_mentions["splitted_mentions"]=rel_mentions["splitted_mentions"].apply(remove_comma)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
G3 = nx.DiGraph()
for r in rel_mentions.iterrows():
    for user in r[1]['splitted_mentions']:
        G3.add_edge(r[1]['username'], user)

In [ ]:
#
nx.write_gexf(G3, path="/content/drive/MyDrive/SMA/g_dir_bbccnnsky.gexf")

In [47]:
in_degrees3=pd.DataFrame(G3.in_degree())

In [ ]:
plt.figure(figsize=(40, 40))
nx.draw_random(G3, cmap=plt.get_cmap('jet'),  
               node_size=(in_degrees3[1]+1)*20, 
               with_labels=True)
plt.show()

In [ ]:
print(f"Ci sono {G3.number_of_nodes()} nodi e {G3.number_of_edges()} archi nel grafo")

Ci sono 767 nodi e 798 archi nel grafo


In [ ]:
in_deg = [val for (node, val) in G3.in_degree()]
print(f"in degree massimo del grafo è {np.max(in_deg)}")   
print(f"in degree massimo del grafo è {np.min(in_deg)}")

in degree massimo del grafo è 180
in degree massimo del grafo è 0


In [ ]:
out_degrees = [val for (node, val) in G3.out_degree()]
print(f"out degree massimo del grafo è {np.max(out_degrees)}")   
print(f"out degree massimo del grafo è {np.min(out_degrees)}")

out degree massimo del grafo è 49
out degree massimo del grafo è 0


In [ ]:
hubs, score =nx.hits(G3, max_iter=50,normalized=True)
  
print("Hub Scores: ", hubs) 
print("Authority Scores: ", score)

Hub Scores:  {'Tian #COVID TERMINATOR#': 1.0074369984770306e-43, 'pfizer': 0.0, 'A': 1.0074369984770306e-43, 'madeinulster': 0.0, 'Jack Rowbotham': 0.0054646598775572545, 'eggman2112': 0.0, 'skynews': 0.0, 'El Martillo de Chele': 0.007457454263323458, 'ruddsbunny': 0.0, 'mrhumbug3': 0.0, 'macpaddy54': 0.0, 'roblev0': 0.0, 'robertcecil1608': 0.0, 'triplegggx': 0.0, 'afewpoints': 0.0, 'macid3000': 0.0, 'gazza41': 0.0, 'beowulfschaefer': 0.0, 'irisheurophile': 0.0, 'cathyby': 0.0, 'johnrya13591294': 0.0, 'susan04071': 0.0, 'david19531501': 0.0, 'chrisgregson123': 0.0, 'markwh2001': 0.0, 'brianbloop2': 0.0, 'davidgsmith18': 0.0, 'davidmetroland': 0.0, 'mindedmusically': 0.0, 'butlerrichard2': 0.0, 'sirtomtucker': 0.0, 'begentle50': 0.0, 'gentlemangeorge': 0.0, 'cheesenbranston': 0.0, 'nicko00127': 0.0, 'dw2essex': 0.0, 'vivcooper': 0.0, 'rosebudorson': 0.0, 'jrwyke': 0.0, 'giles7452': 0.0, 'creativ61543336': 0.0, 'smithdsd11': 0.0, 'edelebear': 0.0, 'andycarp18': 0.0, 'squeakydog1': 0.0, '

In [ ]:
print(f"il coefficente di assortativity {nx.degree_assortativity_coefficient(G3)}")

il coefficente di assortativity -0.26230898454895485


In [ ]:
print(f"il coefficente di reciprocity è {nx.reciprocity(G3)}")

il coefficente di reciprocity è 0.0
